<a href="https://colab.research.google.com/github/samsiroos/SLM-Architecture/blob/main/%DA%86%D8%AA%E2%80%8C%D8%A8%D8%A7%D8%AA_%D8%B3%D8%A7%D8%AF%D9%87_%D8%A8%D8%A7_LSTM_%D8%AF%D8%B1_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np
import os # For file system operations

print("Libraries successfully imported.")

# --- 1. Prepare Training Data ---
# Define simple Persian Question-Answer pairs for the chatbot.
# In a real-world scenario, you would have a much larger dataset.
qa_pairs = [
    ("سلام", "سلام، چطور میتونم کمکتون کنم؟"),
    ("حالت چطوره؟", "من یک هوش مصنوعی هستم و حال ندارم، اما آماده خدمت هستم."),
    ("اسمت چیه؟", "من جیمینی هستم، یک مدل زبانی بزرگ."),
    ("هوش مصنوعی چیست؟", "هوش مصنوعی شاخه‌ای از علوم کامپیوتر است که به ساخت ماشین‌های هوشمند می‌پردازد."),
    ("هوا چطوره؟", "من به اطلاعات آب و هوایی دسترسی ندارم."),
    ("ممنون", "خواهش می‌کنم. خوشحال شدم کمکتون کردم."),
    ("خداحافظ", "خداحافظ، روز خوبی داشته باشید."),
    ("چه کاری میتونی انجام بدی؟", "من میتونم به سوالات شما پاسخ بدم و متن تولید کنم."),
    ("چند سالته؟", "من تاریخ تولد ندارم، چون یک برنامه کامپیوتری هستم."),
    ("کجا زندگی میکنی؟", "من در فضای ابری زندگی میکنم."),
    ("چطور میتونم یاد بگیرم؟", "با مطالعه، تمرین و تکرار میتونید یاد بگیرید."),
    ("اسم تو چیه؟", "من جیمینی هستم."),
    ("کمک", "در چه زمینه‌ای نیاز به کمک دارید؟"),
    ("خوبم", "خوشحالم که حالتون خوبه."),
    ("بله", "بسیار خب."),
    ("نه", "متوجه شدم."),
    ("متشکرم", "خواهش می‌کنم."),
    ("روز بخیر", "روز شما هم بخیر."),
    ("شب بخیر", "شب شما هم بخیر."),
    ("چکار میکنی؟", "من در حال پردازش اطلاعات هستم."),
    ("چه خبر؟", "خبر خاصی نیست، همه چیز خوب است."),
    ("میتونی بهم کمک کنی؟", "بله، حتما. سوالتون رو بپرسید."),
    ("چرا اینجا هستی؟", "من برای کمک به کاربران طراحی شده‌ام."),
    ("تو کی هستی؟", "من یک هوش مصنوعی هستم."),
    ("چه برنامه ای هستی؟", "من یک مدل زبانی بزرگ هستم که توسط گوگل آموزش دیده‌ام."),
    ("میتونی شعر بگی؟", "من میتونم متن تولید کنم، از جمله شعر، اما تخصص اصلی من نیست."),
    ("حوصله ام سر رفته", "میتونم در مورد موضوعات مختلف باهاتون صحبت کنم."),
    ("چه غذایی دوست داری؟", "من هوش مصنوعی هستم و غذا نمیخورم."),
    ("میتونی بهم چیزی یاد بدی؟", "بله، میتونم اطلاعات زیادی رو به شما منتقل کنم."),
    ("چطوری کار میکنی؟", "من با استفاده از شبکه‌های عصبی و داده‌های فراوان آموزش دیده‌ام."),
]

# Add special start and end tokens to target (answer) sequences
# These tokens help the decoder know when to start and stop generating.
input_texts = [pair[0] for pair in qa_pairs]
target_texts = ["_start_ " + pair[1] + " _end_" for pair in qa_pairs]

print(f"تعداد جفت‌های پرسش و پاسخ: {len(qa_pairs)}\n")

# --- Tokenization ---
# Initialize tokenizer. Filters are adjusted to preserve '_' for special tokens like '_start_' and '_end_'.
# Default filters: '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
# We remove '_' from the default filters to keep it as part of the token.
custom_filters = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n' # Removed '_' from the default set
tokenizer = Tokenizer(lower=True, filters=custom_filters)
tokenizer.fit_on_texts(input_texts + target_texts) # Fit on both input and target texts

word_index = tokenizer.word_index # Dictionary of words to their indices
print(f"فهرست کلمات و ایندکس‌های آن‌ها (نمونه):\n{list(word_index.items())[:10]}...\n")

total_words = len(word_index) + 1 # Total unique words + 1 for padding/unknown
print(f"تعداد کل کلمات منحصر به فرد در واژه‌نامه: {total_words}\n")

# Convert texts to sequences of integers
encoder_input_sequences = tokenizer.texts_to_sequences(input_texts)
decoder_input_sequences = tokenizer.texts_to_sequences(target_texts)

# Determine maximum sequence lengths for padding
max_encoder_seq_len = max([len(seq) for seq in encoder_input_sequences])
max_decoder_seq_len = max([len(seq) for seq in decoder_input_sequences])

print(f"حداکثر طول دنباله ورودی (پرسش): {max_encoder_seq_len}")
print(f"حداکثر طول دنباله خروجی (پاسخ): {max_decoder_seq_len}\n")

# Pad sequences to ensure uniform length for model input
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_input_sequences,
                                                                   maxlen=max_encoder_seq_len,
                                                                   padding='post') # Pad with zeros at the end

decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_input_sequences,
                                                                   maxlen=max_decoder_seq_len,
                                                                   padding='post') # Pad with zeros at the end

# Prepare decoder target data (one-hot encoded and shifted by one timestep)
# For example, if target is "_start_ A B _end_", decoder input is "_start_ A B" and target is "A B _end_".
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_len, total_words),
    dtype='float32'
)

for i, seq in enumerate(decoder_input_sequences):
    for t, word_idx in enumerate(seq):
        if t > 0: # Shift target by one timestep
            # decoder_target_data[sample_idx, timestep, word_one_hot_idx] = 1
            decoder_target_data[i, t-1, word_idx] = 1.0

print(f"ابعاد ورودی Encoder (X_encoder): {encoder_input_data.shape}")
print(f"ابعاد ورودی Decoder (X_decoder): {decoder_input_data.shape}")
print(f"ابعاد خروجی Decoder (Y_decoder_target): {decoder_target_data.shape}\n")


# --- Model Filename ---
model_filename = "chatbot_lstm_model.keras" # Using .keras extension for newer TensorFlow versions

# --- Check for existing model and handle training/loading ---
model = None
train_model = True # Default to train if no model exists or user wants to retrain

if os.path.exists(model_filename):
    print(f"\nمدل موجود یافت شد: '{model_filename}'.")
    user_choice = input("آیا می‌خواهید مدل را دوباره آموزش دهید؟ (بله/خیر): ").lower()
    if user_choice == 'خیر':
        train_model = False
        try:
            model = load_model(model_filename)
            print(f"مدل با موفقیت از '{model_filename}' بارگذاری شد.")
        except Exception as e:
            print(f"خطا در بارگذاری مدل: {e}. با آموزش یک مدل جدید ادامه خواهیم داد.")
            train_model = True # Fallback to training if loading fails
    else:
        print("آموزش مجدد توسط کاربر درخواست شد.")
else:
    print(f"\nمدل موجودی یافت نشد در '{model_filename}'. یک مدل جدید آموزش داده خواهد شد.")

if train_model:
    # --- 2. Build the Encoder-Decoder LSTM Model ---
    embedding_dim = 256 # Increased embedding dimension
    lstm_units = 512    # LSTM units

    # --- Encoder ---
    encoder_inputs = Input(shape=(max_encoder_seq_len,))
    encoder_embedding = Embedding(total_words, embedding_dim, name='encoder_embedding')(encoder_inputs)

    # Encoder LSTM 1: Returns sequences and states
    encoder_lstm_1_output_seq, encoder_state_h1, encoder_state_c1 = LSTM(lstm_units, return_sequences=True, return_state=True, name='encoder_lstm_1')(encoder_embedding)

    # Encoder LSTM 2: Returns only its final states (context vectors).
    # It takes the sequence output from encoder_lstm_1 as its input.
    # The initial_state for this LSTM is the states from the first LSTM.
    encoder_outputs, state_h, state_c = LSTM(lstm_units, return_state=True, name='encoder_lstm_2')(encoder_lstm_1_output_seq, initial_state=[encoder_state_h1, encoder_state_c1])

    encoder_states = [state_h, state_c] # The final context states (hidden and cell)

    # --- Decoder ---
    decoder_inputs = Input(shape=(max_decoder_seq_len,))
    decoder_embedding = Embedding(total_words, embedding_dim, name='decoder_embedding')(decoder_inputs)

    # Decoder LSTM 1: Takes encoder's final states as initial_state. Returns sequence and states.
    decoder_lstm_1_layer = LSTM(lstm_units, return_sequences=True, return_state=True, name='decoder_lstm_1')
    decoder_outputs_1_seq, decoder_state_h1, decoder_state_c1 = decoder_lstm_1_layer(decoder_embedding, initial_state=encoder_states)

    # Decoder LSTM 2: Takes sequence output from decoder_lstm_1. Returns sequence and states.
    # Initial states are the states from the first decoder LSTM.
    decoder_lstm_2_layer = LSTM(lstm_units, return_sequences=True, return_state=True, name='decoder_lstm_2')
    decoder_outputs_2_seq, decoder_state_h2, decoder_state_c2 = decoder_lstm_2_layer(decoder_outputs_1_seq, initial_state=[decoder_state_h1, decoder_state_c1])

    decoder_dense = Dense(total_words, activation='softmax', name='decoder_output_dense')
    decoder_outputs = decoder_dense(decoder_outputs_2_seq)

    # Define the training model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.summary()

    # --- 3. Train the Model ---
    epochs = 200 # Increased epochs
    batch_size = 8 # Adjusted batch size for this dataset size

    print("\nشروع آموزش مدل چت‌بات LSTM...")
    history = model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1
    )

    print("\nآموزش مدل به پایان رسید.")
    print(f"دقت نهایی مدل در آموزش: {history.history['accuracy'][-1]:.4f}")
    print(f"خطای نهایی مدل در آموزش: {history.history['loss'][-1]:.4f}")

    # Save the trained model
    model.save(model_filename)
    print(f"مدل با موفقیت در '{model_filename}' ذخیره شد.")
else:
    # If the model was loaded, display its summary
    if model:
        print("\nدر حال استفاده از مدل از پیش موجود.")
        model.summary()
    else:
        print("خطا: مدل نتوانست بارگذاری یا ساخته شود. لطفاً تنظیمات را بررسی کنید.")
        exit()

# --- 4. Build Inference Models (for actual chatting) ---
# Encoder (Inference) Model: Takes input sequence and outputs the encoder's state.
encoder_inputs_inf = Input(shape=(max_encoder_seq_len,))
encoder_embedding_inf = model.get_layer('encoder_embedding')(encoder_inputs_inf)

# Get the LSTM layers from the trained model to use their weights
encoder_lstm_1_inf_layer = model.get_layer('encoder_lstm_1')
encoder_lstm_2_inf_layer = model.get_layer('encoder_lstm_2')

# Encoder inference outputs the final states of both LSTM layers
encoder_lstm_1_output_seq_inf, encoder_state_h1_inf, encoder_state_c1_inf = encoder_lstm_1_inf_layer(encoder_embedding_inf)
encoder_outputs_inf, encoder_state_h2_inf, encoder_state_c2_inf = encoder_lstm_2_inf_layer(encoder_lstm_1_output_seq_inf, initial_state=[encoder_state_h1_inf, encoder_state_c1_inf])

encoder_model = Model(encoder_inputs_inf, [encoder_state_h1_inf, encoder_state_h2_inf, encoder_state_c1_inf, encoder_state_c2_inf]) # Returns all 4 states

# Decoder (Inference) Model: Takes decoder input and previous decoder states, outputs predictions and new states.
decoder_state_input_h1 = Input(shape=(lstm_units,), name='decoder_state_input_h1')
decoder_state_input_c1 = Input(shape=(lstm_units,), name='decoder_state_input_c1')
decoder_state_input_h2 = Input(shape=(lstm_units,), name='decoder_state_input_h2')
decoder_state_input_c2 = Input(shape=(lstm_units,), name='decoder_state_input_c2')
decoder_states_inputs = [decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2]

_decoder_inputs_inf = Input(shape=(1,)) # Decoder input for inference is always a single token at a time
_decoder_embedding_inf = model.get_layer('decoder_embedding')(_decoder_inputs_inf)

# Re-use LSTM layers from the training model
decoder_lstm_1_inf_layer = model.get_layer('decoder_lstm_1')
decoder_lstm_2_inf_layer = model.get_layer('decoder_lstm_2')
decoder_dense_inf_layer = model.get_layer('decoder_output_dense')

# Pass initial states to the first LSTM
decoder_outputs_lstm1_inf, state_h1_new, state_c1_new = decoder_lstm_1_inf_layer(_decoder_embedding_inf, initial_state=[decoder_states_inputs[0], decoder_states_inputs[1]])

# Pass the output sequence from the first LSTM to the second, and its own initial states
decoder_outputs_lstm2_inf, state_h2_new, state_c2_new = decoder_lstm_2_inf_layer(decoder_outputs_lstm1_inf, initial_state=[decoder_states_inputs[2], decoder_states_inputs[3]])

decoder_states_outputs = [state_h1_new, state_c1_new, state_h2_new, state_c2_new] # Collect new states from both LSTMs

_decoder_outputs_inf = decoder_dense_inf_layer(decoder_outputs_lstm2_inf)

decoder_model = Model(
    [_decoder_inputs_inf] + decoder_states_inputs,
    [_decoder_outputs_inf] + decoder_states_outputs
)

# --- Function to decode sequence (generate response) ---
reverse_word_index = dict(map(reversed, word_index.items())) # Map indices back to words

def decode_sequence(input_sentence):
    # Convert input sentence to sequence of integers
    input_seq = tokenizer.texts_to_sequences([input_sentence])[0]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq],
                                                               maxlen=max_encoder_seq_len,
                                                               padding='post')

    # Get the initial states (context vectors) from the encoder
    states_value = encoder_model.predict(input_seq, verbose=0) # Returns [h1, h2, c1, c2]

    # Generate empty target sequence of length 1 (for the start token)
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start token.
    target_seq[0, 0] = word_index['_start_']

    # Initialize decoder states for the first step of decoding
    # Pass all 4 states from the encoder to the decoder's initial states
    decoder_initial_states = states_value # This is now [h1_encoder, h2_encoder, c1_encoder, c2_encoder]

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # Predict next token and new states
        # decoder_model expects [decoder_inputs] + decoder_states_inputs
        # and returns [output_tokens] + new_decoder_states
        output_tokens, h1, c1, h2, c2 = decoder_model.predict([target_seq] + decoder_initial_states, verbose=0)

        # Sample a token (word with highest probability)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_word_index.get(sampled_token_index, '<unk>') # Get word from index, use <unk> for unknown

        # Check for end of sequence or max length
        if sampled_word == '_end_' or len(decoded_sentence.split()) >= max_decoder_seq_len - 1: # -1 for _start_ token
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        # Update the target sequence (of length 1) for the next timestep
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states for the next iteration
        decoder_initial_states = [h1, c1, h2, c2] # Pass the new states from current step to next step

    return decoded_sentence.strip()

# --- Interactive Chat Loop ---
print("\n--- چت‌بات LSTM آماده است! ---")
print("برای خروج، 'خروج' را تایپ کنید.")

while True:
    user_input = input("\nشما: ")
    if user_input.lower() == 'خروج':
        print("چت‌بات: خداحافظ، روز خوبی داشته باشید.")
        break

    # Generate response
    response = decode_sequence(user_input.lower())
    print(f"چت‌بات: {response}")

Libraries successfully imported.
تعداد جفت‌های پرسش و پاسخ: 30

فهرست کلمات و ایندکس‌های آن‌ها (نمونه):
[('_start_', 1), ('_end_', 2), ('من', 3), ('هستم', 4), ('میتونم', 5), ('و', 6), ('به', 7), ('هوش', 8), ('مصنوعی', 9), ('چه', 10)]...

تعداد کل کلمات منحصر به فرد در واژه‌نامه: 173

حداکثر طول دنباله ورودی (پرسش): 5
حداکثر طول دنباله خروجی (پاسخ): 15

ابعاد ورودی Encoder (X_encoder): (30, 5)
ابعاد ورودی Decoder (X_decoder): (30, 15)
ابعاد خروجی Decoder (Y_decoder_target): (30, 15, 173)


مدل موجودی یافت نشد در 'chatbot_lstm_model.keras'. یک مدل جدید آموزش داده خواهد شد.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, 5, 256)    │     44,288 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm_1      │ [(None, 5, 512),  │  1,574,912 │ encoder_embeddin… │
│ (LSTM)              │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 15, 256)   │     44,288 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm_2      │ [(None, 512),     │  2,099,200 │ encoder_lstm_1[0… │
│ (LSTM)              │ (None, 512),      │            │ encoder_lstm_1[0… │
│                     │ (None, 512)]      │            │ encoder_lstm_1[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_1      │ [(None, 15, 512), │  1,574,912 │ decoder_embeddin… │
│ (LSTM)              │ (None, 512),      │            │ encoder_lstm_2[0… │
│                     │ (None, 512)]      │            │ encoder_lstm_2[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_2      │ [(None, 15, 512), │  2,099,200 │ decoder_lstm_1[0… │
│ (LSTM)              │ (None, 512),      │            │ decoder_lstm_1[0… │
│                     │ (None, 512)]      │            │ decoder_lstm_1[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_output_den… │ (None, 15, 173)   │     88,749 │ decoder_lstm_2[0… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,525,549 (28.71 MB)

 Trainable params: 7,525,549 (28.71 MB)

 Non-trainable params: 0 (0.00 B)


شروع آموزش مدل چت‌بات LSTM...
Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 433ms/step - accuracy: 0.0397 - loss: 2.6102
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 412ms/step - accuracy: 0.0693 - loss: 2.3795
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 410ms/step - accuracy: 0.0495 - loss: 2.7008
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 442ms/step - accuracy: 0.0600 - loss: 2.4645
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 650ms/step - accuracy: 0.0950 - loss: 2.3705
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 431ms/step - accuracy: 0.0927 - loss: 2.1199
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 434ms/step - accuracy: 0.1002 - loss: 2.1967
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 557ms/step - accuracy: 0.1052 - loss: 2.0174
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 447ms/step - accuracy: 0.1090 - loss: 2.0522
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 423ms/step - accuracy: 0.1001 - loss: 1.9676
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 639ms/step - accuracy: 0.1025 - loss: 1.9865
Epoch 12/200
4/4 ━━━━━━━━━━━━━━

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1-4243146641.py", line 299, in <cell line: 0>
    user_input = input("\nشما: ")
                 ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1177, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1219, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeEr

TypeError: object of type 'NoneType' has no len()